In [2]:
!pip install soapy


     |████████████████████████████████| 563kB 6.5MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 2.6MB 37.0MB/s 
  Created wheel for soapy: filename=soapy-0.13.2-cp37-none-any.whl size=680158 sha256=42ba1f6cbdb30c3a95d48a6ba9e61116a46a22f7307a1a706e9ba393a035f915
  Stored in directory: /root/.cache/pip/wheels/91/87/8b/e491dbfe9d30ec9ae13bfc96e6c06736f5f12f1b6330a41825
  Created wheel for aotools: filename=aotools-1.0.4-cp37-none-any.whl size=39048 sha256=eea8cc58451ba78792104120bb5563e10129fcdbff828c0765a9eb79fd0dbaf3
  Stored in directory: /root/.cache/pip/wheels/a2/f2/7c/e3024b9bb683e7308f98bd0e127540ed02120b14ccffccdfaa
Successfully built soapy aotools


In [1]:
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np
import matplotlib.pyplot as plt
import yaml
import time

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from soapy import confParse, atmosphere
import soapy

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
help(spaces)

Help on package gym.spaces in gym:

NAME
    gym.spaces

PACKAGE CONTENTS
    box
    dict
    discrete
    multi_binary
    multi_discrete
    space
    tests (package)
    tuple
    utils

CLASSES
    builtins.object
        gym.spaces.space.Space
            gym.spaces.box.Box
            gym.spaces.dict.Dict
            gym.spaces.discrete.Discrete
            gym.spaces.multi_binary.MultiBinary
            gym.spaces.multi_discrete.MultiDiscrete
            gym.spaces.tuple.Tuple
    
    class Box(gym.spaces.space.Space)
     |  Box(low, high, shape=None, dtype=<class 'numpy.float32'>)
     |  
     |  A (possibly unbounded) box in R^n. Specifically, a Box represents the
     |  Cartesian product of n closed intervals. Each interval has the form of one
     |  of [a, b], (-oo, b], [a, oo), or (-oo, oo).
     |  
     |  There are two common use cases:
     |  
     |  * Identical bound for each dimension::
     |      >>> Box(low=-1.0, high=2.0, shape=(3, 4), dtype=np.float32)
  

In [25]:
class AdaptiveOptics(gym.Env):
  def __init__(self,conf_file = None):
    if conf_file:
      self.conf_file = conf_file
    else:
      self.conf_file =  "/content/gdrive/My Drive/ao_prediction/rl/env/conf/sh_8x8.yaml"
    with open(self.conf_file, 'r') as stream:
        self.data_loaded = yaml.safe_load(stream)

    self.action_space = spaces.Box(0, 1, shape=(32,))
    self.observation_space = spaces.Box(-1, 1, shape=(64,64), dtype=np.float32)

  def seed(self):
    pass

  def expert(self):
     if self.sim.config.sim.nDM:
        self.sim.dmCommands[:] = self.sim.recon.reconstruct(self.sim.slopes)
     commands = self.sim.buffer.delay(self.sim.dmCommands, self.sim.config.sim.loopDelay)
     return commands


  def step(self, action):
    loopFrame(self.sim, action)
    next_state = self.sim.sciImgs[0].copy()
    reward = np.sum(next_state)**2/ np.sum(next_state**2)
    return next_state, reward, False, {}

  def reset(self):
    pass

  def render(self):
    self.data_loaded['Atmosphere']['windDirs'] = np.random.randint(0,180,4).tolist()
    self.sim = soapy.Sim(self.conf_file)
    self.sim.aoinit()
    self.sim.makeIMat()
    
  def close(self):
    pass

In [26]:
env = AdaptiveOptics()
env.render()

Loading configuration file...
Pixel Scale: 12.50 pxls/m
subScreenSize: 140 simulation pixels
Loading configuration file... success!
Starting Sim: 2021-07-14-17-09-29
Pixel Scale: 12.50 pxls/m
subScreenSize: 140 simulation pixels
Creating mask...
Turbulence summary @ 500 nm:
| r0 = 0.16 m (0.64" seeing)
| Vbar_5/3 = 11.78 m/s
| tau0 = 4.26 ms
Initialise Infinite Phase Screen 1
Initialise Infinite Phase Screen 2
Initialise Infinite Phase Screen 3
Initialise Infinite Phase Screen 4
Initialising WFSs....
WFS 0: 104 measurements
Initialising 2 DMs...
Making DM Influence Functions...
DM 0: 2 active actuators
Making DM Influence Functions...
DM 1: 30 active actuators
32 total DM Actuators
Initialising Reconstructor...
Initialising 2 Science Cams...
Generating fftw3 plan....
If this takes too long, change fftw_FLAGS.
Done!
Generating fftw3 plan....
If this takes too long, change fftw_FLAGS.
Done!
Initialise Data Storage...
Initialisation Complete!
Making interaction Matrices...
Creating Intera

In [27]:

for i in range(1000):
  next_state, reward, done, _ = env.step(env.expert())
  print(reward)


1 of 10000: sh_8x8  Strehl -- sci_0: inst 0.09, long 0.09 sci_1: inst 0.06, long 0.06 711.2108071882062
2 of 10000: sh_8x8  Strehl -- sci_0: inst 0.10, long 0.09 sci_1: inst 0.06, long 0.05 696.3861510464826
3 of 10000: sh_8x8  Strehl -- sci_0: inst 0.16, long 0.11 sci_1: inst 0.09, long 0.05 660.6038269516249
4 of 10000: sh_8x8  Strehl -- sci_0: inst 0.19, long 0.12 sci_1: inst 0.11, long 0.06 611.2324094138093
5 of 10000: sh_8x8  Strehl -- sci_0: inst 0.20, long 0.14 sci_1: inst 0.11, long 0.07 567.0689799924331
6 of 10000: sh_8x8  Strehl -- sci_0: inst 0.20, long 0.15 sci_1: inst 0.11, long 0.08 533.188342379842
7 of 10000: sh_8x8  Strehl -- sci_0: inst 0.21, long 0.15 sci_1: inst 0.12, long 0.09 503.74518928229094
8 of 10000: sh_8x8  Strehl -- sci_0: inst 0.22, long 0.16 sci_1: inst 0.12, long 0.09 476.93841971499864
9 of 10000: sh_8x8  Strehl -- sci_0: inst 0.23, long 0.17 sci_1: inst 0.12, long 0.09 452.7573719706858
10 of 10000: sh_8x8  Strehl -- sci_0: inst 0.24, long 0.18 sci_

KeyboardInterrupt: ignored

In [ ]:
try:
    import colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !wget https://gist.githubusercontent.com/Tviskaron/4d35eabce2e057dd2ea49a00b00aaa41/raw/f1e25fc6ac6d8f11cb585559ce8b2ab9ffefd67b/colab_render.sh -O colab_render.sh -q
    !sh colab_render.sh
    !wget https://gist.githubusercontent.com/Tviskaron/d91decc1ca5f1b09af2f9f080011a925/raw/0d3474f65b4aea533996ee00edf99a37e4da5561/colab_render.py -O colab_render.py -q 
    import colab_render

In [28]:

import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML
from gym.envs.classic_control import rendering
import matplotlib.pyplot as plt

%matplotlib inline


org_constructor = rendering.Viewer.__init__


def constructor(self, *args, **kwargs):
    org_constructor(self, *args, **kwargs)
    self.window.set_visible(visible=False)


rendering.Viewer.__init__ = constructor


def show_video(folder="./video"):
    mp4list = glob.glob(folder + '/*.mp4')
    if len(mp4list) > 0:
        mp4 = sorted(mp4list, key=lambda x: x[-15:], reverse=True)[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

In [30]:
# env = gym.make(env_name)
env = gym.wrappers.Monitor(env, "./video", force=True)

# generate_session(t_max=10**5, train=False)

env.close()
show_video()

Could not find video


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
rconf_file =  "/content/gdrive/My Drive/ao_prediction/rl/env/conf/sh_8x8.yaml"
with open(conf_file, 'r') as stream:
        data_loaded = yaml.safe_load(stream)
print(data_loaded)

In [ ]:
CONF_FILE = "/content/gdrive/My Drive/ao_prediction/rl/env/conf/sh_8x8.yaml"
def random_config():
    with open(CONF_FILE, 'r') as stream:
        data_loaded = yaml.safe_load(stream)
    data_loaded['Atmosphere']['windDirs'] = np.random.randint(0,180,4).tolist()
    new_file_name =  '/content/gdrive/My Drive/AO_prediction/generator/conf/def.yaml'  
    with open(new_file_name, 'w') as f:
        yaml.dump(data_loaded.copy(), f, default_flow_style=False)
    return new_file_name

def generate(count_data = 100, period = 20, random_conf = False):
    count = 0
    X = []
    Y_sumphase = []
    Y_coefs = []
    
    while count!= count_data:
        if random_conf:
            conf_name = random_config()
        else:
            conf_name = CONF_FILE
        sim = soapy.Sim(conf_name)
        config = soapy.confParse.loadSoapyConfig(conf_name)
        sim.aoinit()
        atmosphere = soapy.atmosphere.atmos(config)
        sim.makeIMat()
        for i in range(1, period+1):
            sim.makeIMat()
            sim.loopFrame()
            
            if i == 1: continue 
            x = None
            y_sum = None
            y_coeffs = None
            y_sum = sim.dms[1].dm_shape
            y_sum = cv2.resize(y_sum, (64, 64)).reshape(1,64,64)

          #  y_sum = (y_sum - np.min(y_sum))/(np.max(y_sum) - np.min(y_sum))
    
        
            x = sim.sciImgs[0].copy().reshape( 1, 64, 64)
            x = (x - np.min(x))/(np.max(x) - np.min(x))
            img = sim.sciImgs[1].copy().reshape(1, 64, 64)
            img = (img - np.min(img))/(np.max(img) - np.min(img))
            x = np.append(x, img, axis=0)
            y_coeffs = sim.dmCommands[sim.dmAct1[1]:sim.dmAct1[1] + sim.dms[1].n_valid_actuators]

            X.append(x)
            Y_sumphase.append(y_sum)
            Y_coefs.append(y_coeffs)
            
            print("\n %d / %d COUN OF DATA \n"%(count, count_data))
            if count== count_data:
                break
            count+=1
    return X, Y_sumphase, Y_coefs


In [ ]:
conf_name = CONF_FILE
sim = soapy.Sim(conf_name)
config = soapy.confParse.loadSoapyConfig(conf_name)
sim.aoinit()
sim.makeIMat()

x = sim.sciImgs[0]
sim.loopFrame()
plt.imshow(x)



In [ ]:
from IPython.display import clear_output
import time
import numpy as np

In [ ]:
X = []

for i in range(100):
  env.step

  if sim.config.sim.nDM:
        sim.dmCommands[:] = sim.recon.reconstruct(sim.slopes)
  commands = sim.buffer.delay(sim.dmCommands, sim.config.sim.loopDelay)

  #commands += np.random.random( commands.shape) 
  loopFrame(sim, commands)
  plt.imshow(x)
  plt.show()
  clear_output(True)


In [ ]:
sim.dmCommands.shape

In [9]:
def loopFrame(sim, new_commands):
    """
    Runs a single from of the entire AO system.

    Moves the atmosphere, runs the WFSs, finds the corrective DM shape and finally runs the science cameras. This can be called over and over to form the "loop"
    """
    # Get next phase screens
    t = time.time()
    sim.scrns = sim.atmos.moveScrns()
    sim.Tatmos = time.time()-t

    # Run Loop...
    ########################################

    # if sim.config.sim.nDM:
    #     sim.dmCommands[:] = sim.recon.reconstruct(sim.slopes)
    # sim.dmCommands = self.buffer.delay(sim.dmCommands, sim.config.sim.loopDelay)
    sim.dmCommands = new_commands
    # Get dmShape from closed loop DMs
    sim.closed_correction = sim.runDM(
            sim.dmCommands, closed=True)

    # Run WFS, with closed loop DM shape applied
    sim.slopes = sim.runWfs(dmShape=sim.closed_correction,
                              loopIter=sim.iters)

    # Get DM shape for open loop DMs
    sim.open_correction = sim.runDM(sim.dmCommands,
                                      closed=False)

    # Pass whole combined DM shapes to science target
    sim.combinedCorrection = sim.open_correction + sim.closed_correction

    sim.runSciCams(sim.combinedCorrection)

    # Save Data
    sim.storeData(sim.iters)

    sim.printOutput(sim.iters, strehl=True)

    sim.addToGuiQueue()

    sim.iters += 1

In [ ]:
 np.random.random( commands.shape) + commands

In [ ]:
commands.shape

In [ ]:
 np.random.random( commands.shape) 

In [ ]:
np.random.random( commands.shape).shape